# List of pathogenic variants for PD
Created: 2018-02-05    
By: Hirotaka

1. Reference clinvar files (2 versions)
2. Pull pathogeic/likely_pathogenic/Conflicting_interpretations_of_pathogenicity
3. Create the combined list of variants of interest from 2 versions
4. Annotate these variants with 2 versions of clinvar
5. Process the two annovar outputs into one.

In [10]:
%%bash
OLD=clinvar_20170501
NEW=clinvar_20180603
OLDF="hg19_"$OLD.txt
NEWF="hg19_"$NEW.txt

grep athogenic $ANNOVAR_DATA/hg19/$OLDF | \
 grep 'arkinson' | \
 grep -v 'Wolff' | \
 grep -v 'Infantile' \
 > $OLD.avinput

grep athogenic $ANNOVAR_DATA/hg19/$NEWF | \
 grep 'arkinson' | \
 grep -v 'Wolff' | \
 grep -v 'Infantile' \
 > $NEW.avinput
 
cat $OLD.avinput $NEW.avinput | cut -f1-5 | sort | uniq > _combined.avinput

table_annovar.pl _combined.avinput $ANNOVAR_DATA/hg19 \
 -buildver hg19 \
 -out $OLD \
 -remove \
 -protocol refGene,cytoBand,avsnp150,$OLD \
 -operation g,r,f,f
 
table_annovar.pl _combined.avinput $ANNOVAR_DATA/hg19 \
 -buildver hg19 \
 -out $NEW \
 -remove \
 -protocol refGene,cytoBand,avsnp150,$NEW \
 -operation g,r,f,f

table_annovar.pl _combined.avinput $ANNOVAR_DATA/hg19 \
 -buildver hg19 \
 -out freq \
 -remove \
 -protocol popfreq_max_20150413 \
 -operation f

-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver hg19 -dbtype refGene -outfile clinvar_20170501.refGene -exonsort _combined.avinput /fdb/annovar/2018-04-16/hg19>
NOTICE: Output files were written to clinvar_20170501.refGene.variant_function, clinvar_20170501.refGene.exonic_variant_function
NOTICE: Reading gene annotation from /fdb/annovar/2018-04-16/hg19/hg19_refGene.txt ... Done with 69265 transcripts (including 16521 without coding sequence annotation) for 28038 unique genes
NOTICE: Processing next batch with 130 unique variants in 130 input lines
NOTICE: Reading FASTA sequences from /fdb/annovar/2018-04-16/hg19/hg19_refGeneMrna.fa ... Done with 69 sequences
-----------------------------------------------------------------
NOTICE: Processing operation=r protocol=cytoBand

NOTICE: Running with system command <annotate_variation.pl -regionanno

In [11]:
import pandas as pd
new = pd.read_table('clinvar_20180603.hg19_multianno.txt')
def get_info(x):
    if x['Func.refGene'] != 'exonic':
        ans = x['Func.refGene']
    else:
        t = x['AAChange.refGene']
        ans = [y for x,y in enumerate(t.split(':')) if ('p.' in y )and (',' not in y)][0]
    return ans
new['AAchange'] = new.apply(get_info, 1)
old = pd.read_table('clinvar_20170501.hg19_multianno.txt')
old['AAchange'] = old.apply(get_info, 1)

In [72]:
old_derived = old[['avsnp150', 'Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene.refGene', 'AAchange', 'CLINSIG', 'CLNDBN']]
new_derived = new[['avsnp150', 'Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene.refGene', 'AAchange', 'CLNSIG', 'CLNDN']]
df = pd.merge(old_derived, new_derived, how='outer', 
              on=['avsnp150', 'Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene.refGene', 'AAchange'],
              suffixes=('_old', '_new'))
def get_ChrOrder(x):
    if x.Chr == 'X':
        ans = 23
    elif x.Chr == 'Y':
        ans = 24
    elif x.Chr == 'MT':
        ans = 25
    else:
        ans = int(x.Chr)
    return ans
df['ChrOrder']=df.apply(get_ChrOrder, 1)
df = df.sort_values(['ChrOrder', 'Start'])
df = df.rename(columns={'CLINSIG':'CLNSIG_1705', 'CLNDBN':'CLNDBN_1705', 'CLNSIG':'CLNSIG_1806', 'CLNDN':'CLNDBN_1806'})
df = df[['avsnp150', 'Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene.refGene', 'AAchange', 
         'CLNSIG_1705', 'CLNSIG_1806', 'CLNDBN_1705', 'CLNDBN_1806']]
df.to_csv('clinvar_1806_1705.csv', index=False)

In [73]:
%%bash
rm *.avinput

In [92]:
# The common ones between potential pathogenics and Meta5
df_pth = pd.read_csv('clinvar_1806_1705.csv')
df_m5 = pd.read_csv('../../GWAS_prog/data/Meta5new.csv')
df_m5 = df_m5[df_m5['Failed final filtering and QC']==0] # 90 SNPs
pd.merge(df_pth, df_m5[['SNP']], left_on='avsnp150', right_on='SNP')

,avsnp150,Chr,Start,End,Ref,Alt,Gene.refGene,AAchange,CLNSIG_1705,CLNSIG_1806,CLNDBN_1705,CLNDBN_1806,SNP
0,rs76763715,1,155205634,155205634,T,C,GBA,p.N409S,Pathogenic|other|other|Pathogenic|Pathogenic|P...,"Pathogenic/Likely_pathogenic,_risk_factor","Gaucher's_disease,_type_1|Parkinson_disease,_l...","Dementia,_Lewy_body,_susceptibility_to|Rigidit...",rs76763715
1,rs34637584,12,40734202,40734202,G,A,LRRK2,p.G2019S,Pathogenic|Pathogenic,Pathogenic,"Parkinson_disease_8,_autosomal_dominant|not_pr...","Inborn_genetic_diseases|Parkinson_disease_8,_a...",rs34637584
